Proof of Concept - Manipulate datasets in the project
===

In [1]:
# Library Utilitaire pour la gestion de projet
import PythonLib

from os.path import splitext, basename

## CSV

In [2]:
import pandas as pd

author= "Mark S. Daskin"
topic = "NETWORK-AND-DISCRETE-LOCATION"

dataset="88-nodes.csv"

dataset_file = "./Datasets/" + author + "/" + dataset

data = pd.read_csv(dataset_file)

In [5]:
data.head()

,ID,LATITUDE,LONGITUDE,STATE POP,CITY POP,FIXED COST,CITY
0,1,73.945478,40.670543,7322564,2819401,189600,< New York NY >
1,2,118.411201,34.112101,3485398,1217405,244500,< Los Angeles CA >
2,3,87.684965,41.837050,2783726,1025174,78700,< Chicago IL >
3,4,95.386728,29.768700,1630553,616877,58000,< Houston TX >
4,5,75.134678,40.006817,1585577,603075,49400,< Philadelphia PA >


## Mongodb

### Create problem

In [11]:
# Generate a connection to our database
import pymongo
from pymongo import MongoClient

mongo_host='ns396089.ip-37-59-38.eu'
mongo_port=32771

client = MongoClient(mongo_host, mongo_port)

# Collection where we wich to store our problem
problems = client[topic][author] # client.[db name].[collection name]

# Prepare the datas to be saved
data.index = data.index.map(str)
record = {
    "metadata": {
        "author": author,
        "topic": topic,
        "dataset": splitext(basename(dataset))[0]
    },
    "data": data.to_dict()
}

# Insert a new entry to the database
problem_id = problems.insert_one(record).inserted_id

### Read problem

In [12]:
# fetch all problems, return an iterator
q = {} # query select the desired intances
f = {} # filter | select the desired fields
records = problems.find(q,f)

In [13]:
list(records)

[{'_id': ObjectId('5ba9a1dd27f7f300226e6e3d')},
 {'_id': ObjectId('5ba9a4da27f7f300226e6e3f')}]

In [15]:
problems.create_index([('metadata.author', pymongo.DESCENDING),
                       ('metadata.topic', pymongo.DESCENDING),
                       ('metadata.dataset', pymongo.DESCENDING)], unique=True)

DuplicateKeyError: E11000 duplicate key error collection: NETWORK-AND-DISCRETE-LOCATION.Mark S. Daskin index: metadata.author_-1_metadata.topic_-1_metadata.dataset_-1 dup key: { : "Mark S. Daskin", : "NETWORK-AND-DISCRETE-LOCATION", : "88-nodes" }